# 非标准化诊疗强基线Baseline-1.65+冲进前二十

本Baseline基于[阿水大佬](https://github.com/datawhalechina/competition-baseline/blob/master/competition/%E7%A7%91%E5%A4%A7%E8%AE%AF%E9%A3%9EAI%E5%BC%80%E5%8F%91%E8%80%85%E5%A4%A7%E8%B5%9B2022/%E9%9D%9E%E6%A0%87%E5%87%86%E5%8C%96%E7%96%BE%E7%97%85%E8%AF%89%E6%B1%82%E7%9A%84%E7%AE%80%E5%8D%95%E5%88%86%E8%AF%8A%E6%8C%91%E6%88%98%E8%B5%9B_baseline.ipynb)开源的baseline上进行优化。

# 改进内容

1. 更换医疗预训练模型MC-BERT：相比使用通用预训练模型，直接使用医疗预训练模型能够触类而长得到更强的基线。↑ [MC-BERT学习资料](https://mp.weixin.qq.com/s?__biz=MzI0NTU5OTM5Nw==&mid=2247483665&idx=1&sn=616596615e201191c043b0037586516b&chksm=e94d5cdbde3ad5cd40dfefb8ec9958d40e07a639976d852e118f6fc36e56bfc842bb720d9a49&token=1618266258&lang=zh_CN#rd)

2. 灵活设计字段拼接方式：使用空格or逗号拼接字段的优点是简单，但句子并不连贯。灵活设计字段拼接，人工增加前缀使得句子语义连贯，在使用[sep]、[unseed]等特殊作用的标志位进行拼接（去年Top3均采用了此方式）会有明显的提升。↑ [点击观看2021年Top3答辩视频](https://www.bilibili.com/video/BV1hq4y1r7xB?p=15&vd_source=4cb6c45950d255204ceab92e062fd928)

3. CLS or avgpooling ：平均池化BERT模型最后一层的特征,相较于直接使用CLS位置的特征可能效果更好，（部分比赛ticks分享中，融合BERT后4层的向量也可能提升下游任务性能）。↑

4. 遵循赛题要求使用marco-f1和mirco-f1评测模型：使用Accuracy评测模型效果线下效果和线上效果差异过大。↑（线下1.68，线上1.65）

# 后续优化

- 对抗训练FGM、PGM 学习资料[对抗训练](https://github.com/Zeta-THU/NLP-Interview-Notes/blob/9a978660bbb2009956a49067437ec945c10abed8/Trick/SmallSampleProblem/AdversarialTraining/AdversarialTraining.md)

- 不平衡类别损失函数 focal_loss、Dice_loss、DSC Loss 学习资料[NLP-Loss-Pytorch](https://github.com/shuxinyin/NLP-Loss-Pytorch)

- RDrop对比学习 学习资料[R-Drop：提升有监督任务性能最简单的方法](https://zhuanlan.zhihu.com/p/418305402)

- 设计新的损失函数，考虑label_i和label_j之间的层次关系

- 融合主体/属性/客体等三元组知识

In [1]:
from tqdm import tqdm
import pandas as pd

In [2]:
train_df = pd.read_excel('非标准化疾病诉求的简单分诊挑战赛2/data_train.xlsx')
test_df = pd.read_excel('非标准化疾病诉求的简单分诊挑战赛2/data_test.xlsx')
test_submit = pd.read_csv('data/提交示例.csv')
train_df.shape

(22868, 8)

In [3]:
# 原始 : 简单拼接
train_text = train_df['diseaseName'] + ' ' + train_df['conditionDesc'] + ' ' + train_df['title'] + ' ' + train_df['hopeHelp']

In [4]:
# 优化拼接方法 : 清洗噪声符号 + 人工构建字段描述模板 + 去重重复字段 + 使用vocab.txt中的[unused01]连接各字段
def clean_str(x):
    return x.replace('\r', '').replace('\t', ' ').replace('\n', ' ')

merge_col = ["diseaseName","conditionDesc","title","hopeHelp"]
for col in merge_col:
    train_df[col].fillna('', inplace=True)
    test_df[col].fillna('', inplace=True)
    train_df[col] = train_df[col].apply(lambda x: clean_str(x))
    test_df[col] = test_df[col].apply(lambda x: clean_str(x))

sentences = []
for idx,rows in train_df.iterrows():
    merge_col_template = {"diseaseName":"疾病名称:","conditionDesc":"症状描述:","title":"","hopeHelp":""}
    if rows["title"] in rows["conditionDesc"] or rows["title"] in rows["diseaseName"] or rows["title"] == rows["hopeHelp"] or len(rows["title"]) == 0:
        merge_col_template.pop("title")
    if  len(rows["hopeHelp"]) == 0:
        merge_col_template.pop("hopeHelp")
    sentence = "[unused01]".join([str(temp) + str(rows[col]) for col,temp in merge_col_template.items()])
    sentences.append(sentence)
train_df["sentence"] = sentences

sentences = []
for idx,rows in test_df.iterrows():
    merge_col_template = {"diseaseName":"疾病名称:","conditionDesc":"症状描述:","title":"","hopeHelp":""}
    if rows["title"] in rows["conditionDesc"] or rows["title"] in rows["diseaseName"] or rows["title"] == rows["hopeHelp"] or len(rows["title"]) == 0:
        merge_col_template.pop("title")
    if  len(rows["hopeHelp"]) == 0:
        merge_col_template.pop("hopeHelp")
    sentence = "[unused01]".join([str(temp) + str(rows[col]) for col,temp in merge_col_template.items()])
    sentences.append(sentence)
test_df["sentence"] = sentences

# 简单比较一下拼接后的句子
print('简单拼接:')
print(train_text[1])
print('优化拼接：')
print(train_df['sentence'][1])

简单拼接:
右膝前交叉韧带断裂，半月板三度重度损伤 右膝受力小腿骨和大腿骨就会滑开，走路会有隐隐作痛 大概做韧带重建手术需要多少费用？ 大概做韧带重建手术需要多少费用？
优化拼接：
疾病名称:右膝前交叉韧带断裂，半月板三度重度损伤[unused01]症状描述:右膝受力小腿骨和大腿骨就会滑开，走路会有隐隐作痛[unused01]大概做韧带重建手术需要多少费用？


In [5]:
from sklearn.model_selection import train_test_split
from transformers import BertModel,BertTokenizer
pretrain_dir = r'PretrainModel/mc_bert_base'
tokenizer = BertTokenizer.from_pretrained(pretrain_dir,additional_special_tokens=['[unused99]'])
# 处理label j 无标记数据，拼接在乱序数据集后面
unlabeled_train_df = train_df[train_df['label_j']==-1]
print(unlabeled_train_df.shape)
train_df = train_df[train_df['label_j']!=-1]
# 划分数据集
train_text,val_text,_,_ = train_test_split(train_df,train_df['id'],test_size=0.1,random_state=42)
print(train_text.shape)
train_text = pd.concat([train_text.sample(frac=1.0,random_state=42),unlabeled_train_df],axis=0)
print(train_text.shape)
train_encoding = tokenizer(train_text['sentence'].tolist(), truncation=True, padding='max_length', max_length=256)
val_encoding = tokenizer(val_text['sentence'].tolist(), truncation=True, padding='max_length', max_length=256)
test_encoding = tokenizer(test_df['sentence'].tolist(), truncation=True, padding='max_length', max_length=256)

(8746, 9)
(12709, 9)
(21455, 9)


In [6]:
# 查看编码
tokenizer.decode(tokenizer("疾病描述:右膝前交叉韧带断裂[unused99]症状描述:右膝受力小腿骨和大腿骨就会滑开[unused99]大概做韧带重建手术需要多少费用？",truncation=True, padding='max_length', max_length=256)["input_ids"])

'[CLS] 疾 病 描 述 : 右 膝 前 交 叉 韧 带 断 裂 [unused99] 症 状 描 述 : 右 膝 受 力 小 腿 骨 和 大 腿 骨 就 会 滑 开 [unused99] 大 概 做 韧 带 重 建 手 术 需 要 多 少 费 用 ？ [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [

In [7]:
import numpy as np
import random
import torch

def set_seed(seed):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

set_seed(42)

In [8]:
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch import nn
# 数据集读取
class XunFeiDataset(Dataset):
    def __init__(self, encodings, label_i, label_j):
        self.encodings = encodings
        self.label_i = label_i
        self.label_j = label_j
    # 读取单个样本
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['label_i'] = torch.tensor(int(self.label_i[idx]))
        item['label_j'] = torch.tensor(int(self.label_j[idx]))
        return item
    def __len__(self):
        return len(self.label_i)

train_dataset = XunFeiDataset(train_encoding,
                              train_text['label_i'].values,
                              train_text['label_j'].values)
val_dataset = XunFeiDataset(val_encoding,
                            val_text['label_i'].values,
                            val_text['label_j'].values)
test_dataset = XunFeiDataset(test_encoding,
                             [0] * len(test_df),
                             [0] * len(test_df))

In [9]:
# 单个读取到批量读取
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=False)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [10]:
class XunFeiModel(nn.Module):
    def __init__(self, num_labels_i, num_labels_j):
        super(XunFeiModel,self).__init__()
        #Load Model with given checkpoint and extract its body
        self.model = BertModel.from_pretrained(pretrain_dir)
        self.dropout = nn.Dropout(0.1)
        self.classifier_i = nn.Linear(768, num_labels_i)
        self.classifier_j = nn.Linear(768, num_labels_j)

    def forward(self, input_ids=None, attention_mask=None,labels=None):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        # outputs是一个包含两个输出的元组
        # outputs[0] = last hidden state 输出形状:(batch_size,max_len,768)
        # outputs[1] = pooing hidden state 输出形状:(batch_size,768)
        sequence_output = self.dropout(outputs[1])
        # 阿水大佬原版 sequence_output[:,0,:].view(-1,768) 相当于取CLS的句向量 输出形状：(batch_size,768)
        logits_i = self.classifier_i(sequence_output) # 输出形状：(batch_size,num_labels_i)
        logits_j = self.classifier_j(sequence_output) # 输出形状：(batch_size,num_labels_j)

        return logits_i, logits_j

In [11]:
class_i_nums = len(train_text['label_i'].unique()) # 20
class_j_nums = len(train_text['label_j'].unique()) - 1  # 61
model = XunFeiModel(class_i_nums, class_j_nums)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = 'cpu' or 'gpu'
model = model.to(device)

Some weights of the model checkpoint at E:\HuProject\NLPsource\PretrainModel\mc_bert_base were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
from torch.nn import CrossEntropyLoss
from sklearn.metrics import f1_score
from transformers import AdamW,get_linear_schedule_with_warmup

param_optimizer = list(model.named_parameters())
no_decay = ["bias", 'LayerNorm.bias', "LayerNorm.weight", 'layer_norm.bias', 'layer_norm.weight']
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
    {
        "params": [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0
    },
]
# 设置学习率
optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5)
loss_fn = CrossEntropyLoss()

In [13]:
def train(epochs=10):
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0.0, num_training_steps=len(train_dataloader) * epochs)
    print("train ...")
    total_iter_num = 0
    best_score = 0.0
    total_iter = len(train_dataloader) * epochs
    for epoch in range(epochs):
        model.train()
        iter_num = 0
        total_train_loss = 0
        pred_is,pred_js = [],[]
        label_is,label_js = [],[]
        for batch in train_dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            label_i = batch['label_i'].to(device)
            label_j = batch['label_j'].to(device)
            pred_i, pred_j = model(input_ids,attention_mask)

            # 疾病方向标签-1时，不记录损失
            valid = label_j != -1
            if len(pred_j[valid]) != 0:
                loss = loss_fn(pred_i, label_i)  + loss_fn(pred_j[valid], label_j[valid])
            else:
                loss = loss_fn(pred_i, label_i)
            # 反向梯度信息
            loss.backward()
            # 参数更新
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

            iter_num += 1
            total_iter_num += 1

            pred_is.extend(pred_i.argmax(1).data.cpu().numpy())
            label_is.extend(label_i.data.cpu().numpy())

            if len(pred_j[valid]) != 0:
                pred_js.extend(pred_j[valid].argmax(1).data.cpu().numpy())
                label_js.extend(label_j[valid].data.cpu().numpy())

            total_train_loss += loss.item()

            if(total_iter_num % 50 == 0):
                lis = f1_score(y_true=pred_is,y_pred=label_is,average='macro')
                ljs = f1_score(y_true=pred_js,y_pred=label_js,average='micro')
                print("iter_num: %d, step:%.2f%% , loss: %.4f,  label_i:%.4f label_j:%.4f score:%.4f" % (
                    total_iter_num , total_iter_num  / total_iter, total_train_loss/iter_num ,
                    lis,
                    ljs,
                    (lis + ljs)
                ))
            # 验证
            if(total_iter_num % 200 == 0):
                _,_,score = validation()
                if best_score < score:
                    print("score: %.4f -----> %.4f" % (best_score,score))
                    best_score = score
                    print("保存模型")
                    model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self
                    # output_model_file = 'model_{:.4f}_{}.bin'.format(best_score,str(iter_num))
                    output_model_file = 'model_best.bin'
                    torch.save(model_to_save, output_model_file)

    return best_score

def validation():
    model.eval()
    pred_is,pred_js = [],[]
    label_is,label_js = [],[]
    for batch in val_dataloader:
        with torch.no_grad():
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            label_i = batch['label_i'].to(device)
            label_j = batch['label_j'].to(device)

            pred_i, pred_j = model(
                input_ids,
                attention_mask
            )
            pred_is.extend(pred_i.argmax(1).data.cpu().numpy())
            pred_js.extend(pred_j.argmax(1).data.cpu().numpy())
            label_is.extend(label_i.data.cpu().numpy())
            label_js.extend(label_j.data.cpu().numpy())

    lis = f1_score(y_true=pred_is,y_pred=label_is,average='macro')
    ljs = f1_score(y_true=pred_js,y_pred=label_js,average='micro')

    print("-------------------------------")
    print("label_i:%.4f label_j:%.4f score:%.4f" % (lis,ljs,(lis + ljs)))
    print("-------------------------------")
    model.train()
    return lis,ljs,(lis + ljs)

def prediction():
    model.eval()
    test_label_i = []
    test_label_j = []
    for batch in tqdm(test_dataloader):
        with torch.no_grad():
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            pred_i, pred_j = model(input_ids, attention_mask)
            test_label_i += list(pred_i.argmax(1).data.cpu().numpy())
            test_label_j += list(pred_j.argmax(1).data.cpu().numpy())
    return test_label_i, test_label_j

In [14]:
# 开始训练，学习率设置较小，训练轮次尽量大一些
epochs = 10
best_score = train(epochs)

train ...
iter_num: 50, step:0.00% , loss: 6.5798,  label_i:0.2255 label_j:0.0925 score:0.3180
iter_num: 100, step:0.01% , loss: 5.8392,  label_i:0.4513 label_j:0.1931 score:0.6444
iter_num: 150, step:0.01% , loss: 5.2768,  label_i:0.5456 label_j:0.2692 score:0.8148
iter_num: 200, step:0.01% , loss: 4.8503,  label_i:0.6138 label_j:0.3162 score:0.9300
-------------------------------
label_i:0.7869 label_j:0.5025 score:1.2894
-------------------------------
score: 0.0000 -----> 1.2894
保存模型
iter_num: 250, step:0.02% , loss: 4.5259,  label_i:0.6555 label_j:0.3560 score:1.0115
iter_num: 300, step:0.02% , loss: 4.2683,  label_i:0.6843 label_j:0.3794 score:1.0637
iter_num: 350, step:0.03% , loss: 4.0560,  label_i:0.7031 label_j:0.4007 score:1.1038
iter_num: 400, step:0.03% , loss: 3.8863,  label_i:0.7172 label_j:0.4142 score:1.1314
-------------------------------
label_i:0.8213 label_j:0.5414 score:1.3627
-------------------------------
score: 1.2894 -----> 1.3627
保存模型
iter_num: 450, step:0.0

In [15]:
"最优线下成绩：",best_score

('最优线下成绩：', 1.684131238480608)

In [16]:
# 加载最佳模型，提交文件
model = torch.load('model_best.bin')
test_pred_i, test_pred_j = prediction()
test_submit['label_i'] = test_pred_i
test_submit['label_j'] = test_pred_j
test_submit.to_csv('bert_submit.csv', index=None)

100%|██████████| 475/475 [00:49<00:00,  9.53it/s]


In [18]:
"线上测试成绩:1.65+ / "

'线上测试成绩:1.65+ '

| base-model  | score   |
|-------------|---------|
| mc-bert     | 1.6507  |